### Import data

In [13]:
import pandas as pd

features = pd.read_csv('databases/3m_training.csv')
features.head(5)

,outperf vs index 1m,outperf vs index 3m,ST momentum,LT momentum,Index vs max,Index st momentum,ebit vs peak,net income vs peak,market cap vs peak,Margin vs peak,PE,xEbit,xSales,PE vs peak,xEbit vs peak,xSales vs peak,ND/market cap,outperf next 3 months
0,0.007191,-0.083593,0.076685,-0.472196,-0.2,0.017742,0.000000,0.000000,-0.042669,-0.016447,11.096157,8.764507,0.390003,-0.195909,-0.040863,-0.047339,0.046168,0
1,-0.017525,0.071289,0.009150,-0.346791,-0.2,0.017742,-0.266321,-0.496339,-0.051437,-0.247309,17.685318,8.772884,0.592203,-0.051437,-0.043755,-0.218866,0.185074,0
2,-0.002132,0.001588,0.024445,0.272849,-0.2,0.017742,0.000000,0.000000,-0.035756,0.000000,19.453475,17.377210,0.837097,-0.235442,-0.168420,-0.027351,0.318709,0
3,0.044802,0.041664,0.204800,0.556955,-0.2,0.017742,0.000000,0.000000,0.000000,-0.045119,12.874137,10.429705,0.975562,0.000000,-0.008045,-0.052801,0.498034,0
4,0.001149,0.041071,0.026255,0.251466,-0.2,0.017742,0.000000,0.000000,-0.032763,0.000000,8.221638,4.630687,0.691503,-0.066863,-0.132443,-0.060985,0.141099,0


### process database

In [14]:
#Process database
import numpy as np

#save labels
labels = np.array(features['outperf next  3 months'])
#remove labels
features=features.drop('outperf next  3 months', axis = 1)
#save features names
feature_list = list(features.columns)
#convert to numpy array
features = np.array(features)

### Split database in training and cross validation sets

In [15]:
from sklearn.model_selection import train_test_split
#stratify = labels allows for consistant class distribution between sets
X_train, X_cv, y_train, y_cv = train_test_split(features, labels, stratify = labels)


### Show distribution


In [16]:
train_zeros = (y_train == 0).sum()
train_ones = (y_train == 1).sum()
train_pct = train_ones / (train_zeros + train_ones)
print('Percentage of ones in train set is', train_pct )

cv_zeros = (y_cv == 0).sum()
cv_ones = (y_cv == 1).sum()
cv_pct = cv_ones / (cv_zeros + cv_ones)
print('Percentage of ones in cv set is', cv_pct )


Percentage of ones in train set is 0.40494044993383327
Percentage of ones in cv set is 0.40476190476190477


### Setup Grid search

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix

import matplotlib.pyplot as plt
plt.style.use("ggplot")


clf = RandomForestClassifier(n_jobs=-1)

param_grid = {
    'min_samples_split': [2, 5, 10, 20, 40], 
    'n_estimators' : [50, 100, 200, 400],
    'max_depth': [3, 5, 15, 25],
    'max_features': [3, 5, 10, 17]
}

scorers = {
    'precision_score': make_scorer(precision_score),
    'recall_score': make_scorer(recall_score),
    'accuracy_score': make_scorer(accuracy_score)
}



def grid_search_wrapper(refit_score='accuracy_score'):
    """
    fits a GridSearchCV classifier using refit_score for optimization
    prints classifier performance metrics
    """
    skf = StratifiedKFold(n_splits=10)
    grid_search = GridSearchCV(clf, param_grid, scoring=scorers, refit=refit_score,
                           cv=skf, return_train_score=True, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # make the predictions
    y_pred = grid_search.predict(X_cv)

    print('Best params for {}'.format(refit_score))
    print(grid_search.best_params_)

    # confusion matrix on the cv data.
    print('\nConfusion matrix of Random Forest optimized for {} on the cv data:'.format(refit_score))
    print(pd.DataFrame(confusion_matrix(y_cv, y_pred),
                 columns=['pred_neg', 'pred_pos'], index=['neg', 'pos']))
    return grid_search



### Run grid search

In [18]:
grid_search_clf = grid_search_wrapper(refit_score='accuracy_score')


Best params for accuracy_score
{'max_depth': 25, 'max_features': 17, 'min_samples_split': 2, 'n_estimators': 200}

Confusion matrix of Random Forest optimized for accuracy_score on the cv data:
     pred_neg  pred_pos
neg       375        75
pos       134       172


### Display results

In [20]:
results = pd.DataFrame(grid_search_clf.cv_results_)
results.to_csv('tuning reports/rf_tuning.csv')
results = results.sort_values(by='mean_test_recall_score', ascending=False)
results[['mean_test_precision_score', 'mean_test_recall_score', 'mean_test_accuracy_score', 'param_max_depth', 'param_max_features', 'param_min_samples_split', 'param_n_estimators']].round(3).head()


,mean_test_precision_score,mean_test_recall_score,mean_test_accuracy_score,param_max_depth,param_max_features,param_min_samples_split,param_n_estimators
302,0.687,0.595,0.724,25,17,2,200
282,0.680,0.578,0.716,25,10,2,200
283,0.685,0.577,0.719,25,10,2,400
307,0.682,0.576,0.718,25,17,5,400
305,0.669,0.573,0.711,25,17,5,100
